In [62]:
from Bio import Entrez
import numpy as np

def search(query):
    Entrez.email = 'your.email@example.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='most recent', 
                            retmax='30',
                            retmode='xml', 
                            reldate = 7, #only within n days from now
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'your.email@example.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

search_results = search('(Biomech*[Title/Abstract] OR locomot*[Title/Abstract])')
id_list = search_results['IdList']
papers = fetch_details(id_list)
print("")
titles = [0 for i in enumerate(papers['PubmedArticle'])]

for i, paper in enumerate(papers['PubmedArticle']):
    titles[i] = papers['PubmedArticle'][i]['MedlineCitation']['Article']['ArticleTitle']

p_num = 0
for i, paper in enumerate(papers['PubmedArticle']):
    print("* [%s](https://www.ncbi.nlm.nih.gov/pubmed/%s)" % (paper['MedlineCitation']['Article']['ArticleTitle'],paper['MedlineCitation']['PMID']))
    for auth in np.arange(1,np.size(paper['MedlineCitation']['Article']['AuthorList'])):
        try:
            print("%s %s," % (paper['MedlineCitation']['Article']['AuthorList'][auth]['LastName'],paper['MedlineCitation']['Article']['AuthorList'][auth]['Initials']))
        except:
            print('NAME ERROR')
    print('%s' % (paper['MedlineCitation']['Article']['Journal']['Title']) )
    p_num = p_num +1
    # #store keywords 
    # print("----")
    # if paper['MedlineCitation']['KeywordList'] != []:
    #     for kw in np.arange(1,np.size(paper['MedlineCitation']['KeywordList'][0])):
    #         print(paper['MedlineCitation']['KeywordList'][0][kw-1][:])
    # else:
    #     print("NO_KEYWORDS")
    print("<br>  ") #linebreak for github md 
    #end keywords test
    print("")

print(p_num)


* [Experimental cervical interspinous ligament pain altered cervical joint motion during dynamic extension movement.](https://www.ncbi.nlm.nih.gov/pubmed/30991232)
Lindstrøm R,
Graven-Nielsen T,
Hirata RP,
Clinical biomechanics (Bristol, Avon)
<br>  

* [Studies on long term behavioural changes in group-housed rat models of brain and spinal cord injury using an automated home cage recording system.](https://www.ncbi.nlm.nih.gov/pubmed/30991030)
Chapman GE,
Sillito RR,
Ip THR,
Akhigbe G,
Becker SC,
Price AW,
Michael-Titus AT,
Armstrong JD,
Tremoleda JL,
Journal of neuroscience methods
<br>  

* [In vivo three-dimensional kinematics of the cervical spine during maximal active head rotation.](https://www.ncbi.nlm.nih.gov/pubmed/30990826)
Chen G,
Zhai X,
He X,
PloS one
<br>  

* [Oxytocin blocks enhanced motivation for alcohol in alcohol dependence and blocks alcohol effects on GABAergic transmission in the central amygdala.](https://www.ncbi.nlm.nih.gov/pubmed/30990816)
Kirson D,
Zallar 

In [65]:
import csv

myfile =  open('literature_update.csv', 'w')
wr = csv.writer(myfile)
for item in titles:
    wr.writerow([item])

* [Experimental cervical interspinous ligament pain altered cervical joint motion during dynamic extension movement.](https://www.ncbi.nlm.nih.gov/pubmed/30991232)
Lindstrøm R,
Graven-Nielsen T,
Hirata RP,
Clinical biomechanics (Bristol, Avon)
<br>  

* [In vivo three-dimensional kinematics of the cervical spine during maximal active head rotation.](https://www.ncbi.nlm.nih.gov/pubmed/30990826)
Chen G,
Zhai X,
He X,
PloS one
<br>  

* [Report on Three Porcine Proof-of-concept Studies: Comparison of a Dermatome With a Rotating Excision Ring With Conventional Dermatomes for the Harvesting of STSGs and Excision of Necrosis.](https://www.ncbi.nlm.nih.gov/pubmed/30990780)
Pittinger T,
Bailey K,
Powell HM,
Wounds : a compendium of clinical research and practice
<br>  

* [Bio-inspired photo-detachable dry self-cleaning surface.](https://www.ncbi.nlm.nih.gov/pubmed/30990696)
Li X,
Ma J,
Zhang R,
Qin F,
Wang J,
Hu TS,
Zhang Y,
Xu Q,
Langmuir : the ACS journal of surfaces and colloids
<br>  

* [Sex and Laterality Differences in Medullary Humerus Morphology.](https://www.ncbi.nlm.nih.gov/pubmed/30989818)
Tashjian RZ,
Henninger HB,
Bachus KN,
Anatomical record (Hoboken, N.J. : 2007)
<br>  

* [Comparison of Less Invasive Stabilization System Plate and Retrograde Intramedullary Nail in the Fixation of Femoral Supracondylar Fractures in the Elderly: A Biomechanical Study.](https://www.ncbi.nlm.nih.gov/pubmed/30989797)
Ma JX,
Wang S,
Sun L,
Wang Y,
Lu B,
Bai HH,
Hu YC,
Ma XL,
Orthopaedic surgery
<br>  

* [Biomechanical properties of red blood cells infected by Plasmodium berghei ANKA.](https://www.ncbi.nlm.nih.gov/pubmed/30989677)
Lee DH,
Han SJ,
Yang W,
Quan FS,
Kim KS,
Journal of cellular physiology
<br>  

* [Biomechanical comparison of meniscus-suture constructs for pullout repair of medial meniscus posterior root tears.](https://www.ncbi.nlm.nih.gov/pubmed/30989353)
Mae T,
Tachibana Y,
Iuchi R,
Nakata K,
Yamashita T,
Shino K,
Journal of experimental orthopaedics
<br>  

* [The relationships between the quadratus plantae and the flexor digitorum longus and the flexor hallucis longus.](https://www.ncbi.nlm.nih.gov/pubmed/30989352)
Takabayashi T,
Inai T,
Kikumoto T,
Hirabayashi R,
Ito W,
Nakamura E,
Ikezu M,
Kaneko F,
Kageyama I,
Surgical and radiologic anatomy : SRA
<br>  

* [Effect of coadministration of the GABA<sub>B</sub> agonist baclofen and the 5-HT<sub>2C</sub> agonist Ro60-0175 on the expression of amphetamine-induced locomotor sensitization.](https://www.ncbi.nlm.nih.gov/pubmed/30989246)
Ruíz-García RI,
Jiménez JC,
Miranda F,
Experimental brain research
<br>  

* [Contribution of biomimetic collagen-ligand interaction to intrafibrillar mineralization.](https://www.ncbi.nlm.nih.gov/pubmed/30989106)
Jiao K,
Tonggu L,
Wang LG,
Zhang SL,
Yang YD,
Zhang L,
Bian JH,
Hao DX,
Wang CY,
Ma YX,
Arola DD,
Breschi L,
Chen JH,
Tay FR,
Niu LN,
Science advances
<br>  

* [Inter-Session Reliability of the Tennis Serve and Influence of the Laboratory Context.](https://www.ncbi.nlm.nih.gov/pubmed/30988840)
Forthomme B,
Croisier JL,
Brüls O,
Denoël V,
Paulus J,
Schwartz C,
Journal of human kinetics
<br>  

* [Individual play patterns stimulated by a familiar object are group-driven.](https://www.ncbi.nlm.nih.gov/pubmed/30988518)
Rehn T,
de Oliveira D,
Malovrh Š,
Keeling L,
Scientific reports
<br>  

* [The evolutionary history of the human face.](https://www.ncbi.nlm.nih.gov/pubmed/30988489)
Stringer CB,
Kimbel WH,
Wood B,
Harvati K,
O'Higgins P,
Bromage TG,
Arsuaga JL,
Nature ecology & evolution
<br>  

* [Sexually dimorphic regulation of behavioral states by dopamine in <i>Caenorhabditis elegans</i>.](https://www.ncbi.nlm.nih.gov/pubmed/30988167)
Harada K,
Matsuda S,
Kyo K,
Wang M,
Maruyama K,
Awaji T,
Tsuboi T,
The Journal of neuroscience : the official journal of the Society for Neuroscience
<br>  

* [Inhibition of the deubiquitinase USP8 corrects a Drosophila PINK1 model of mitochondria dysfunction.](https://www.ncbi.nlm.nih.gov/pubmed/30988163)
Sanchez-Martinez A,
Corrà S,
Chakraborty J,
Marchesan E,
Locatello L,
Da Rè C,
Cusumano P,
Caicci F,
Ferrari V,
Costa R,
Bubacco L,
Rasotto MB,
Szabo I,
Whitworth AJ,
Scorrano L,
Ziviani E,
Life science alliance
<br>  

* [A Biomechanical Comparison of Alternative Graft Preparations for All-Inside Anterior Cruciate Ligament Reconstruction.](https://www.ncbi.nlm.nih.gov/pubmed/30987907)
Tsouris ND,
Hassan CR,
Elbayar JH,
Qin YX,
Komatsu DE,
Rizzi AV,
Paci JM,
Arthroscopy : the journal of arthroscopic & related surgery : official publication of the Arthroscopy Association of North America and the International Arthroscopy Association
<br>  

* [A biomechanical study of gait initiation in Down syndrome.](https://www.ncbi.nlm.nih.gov/pubmed/30987596)
Cimolin V,
Capodaglio P,
Condoluci C,
Galli M,
BMC neurology
<br>  

* [Motion of the distal tibiofibular syndesmosis under different loading patterns: A biomechanical study.](https://www.ncbi.nlm.nih.gov/pubmed/30987529)
Chen DW,
Li B,
Yang YF,
Yu GR,
Journal of orthopaedic surgery (Hong Kong)
<br>  

* [Exploring the Role of Wearable Technology in Sport Kinematics and Kinetics: A Systematic Review.](https://www.ncbi.nlm.nih.gov/pubmed/30987014)
Papi E,
McGregor AH,
Sensors (Basel, Switzerland)
<br>  

* [Finite Element Analysis and Biomechanical Testing of the New MiniMIS Short Stem.](https://www.ncbi.nlm.nih.gov/pubmed/30986878)
Huber D,
Sonntag R,
Kretzer JP,
Zeitschrift fur Orthopadie und Unfallchirurgie
<br>  

* [The effect of limb dominance on change of direction biomechanics: A systematic review of its importance for injury risk.](https://www.ncbi.nlm.nih.gov/pubmed/30986764)
Bishop C,
Thomas C,
Comfort P,
Jones PA,
Physical therapy in sport : official journal of the Association of Chartered Physiotherapists in Sports Medicine
<br>  

* [Comparison of finite helical axes of normal and anatomically designed prosthetic knees.](https://www.ncbi.nlm.nih.gov/pubmed/30986763)
Tomita T,
Yamazaki T,
Oda K,
Nakajima M,
Nakane K,
Kono K,
Fujito T,
Yoshikawa H,
Sugamoto K,
Clinical biomechanics (Bristol, Avon)
<br>  

* [Skipping has lower knee joint contact forces and higher metabolic cost compared to running.](https://www.ncbi.nlm.nih.gov/pubmed/30986589)
Zwetsloot KA,
Houmard J,
DeVita P,
Gait & posture
<br>  

* [Unsupervised gait retraining using a wireless pressure-detecting shoe insole.](https://www.ncbi.nlm.nih.gov/pubmed/30986588)
Lippmann K,
Shakoor N,
Ferrigno C,
Wimmer MA,
Gait & posture
<br>  

* [Differences in temporal gait mechanics are associated with decreased perceived ankle joint health in individuals with chronic ankle instability.](https://www.ncbi.nlm.nih.gov/pubmed/30986587)
Terada M,
McCann R,
Thomas A,
Johnson N,
Gribble P,
Gait & posture
<br>  

* [Predictors of Healing Ligament Size and Magnetic Resonance Signal Intensity at 6 Months After Bridge-Enhanced Anterior Cruciate Ligament Repair.](https://www.ncbi.nlm.nih.gov/pubmed/30986359)
Kiapour AM,
Kalish LA,
Ecklund K,
NAME ERROR
Fleming BC,
Freiberger C,
Henderson R,
Kramer D,
Micheli L,
Yen YM,
The American journal of sports medicine
<br>  

* [Visual-Spatial Memory Deficits Are Related to Increased Knee Valgus Angle During a Sport-Specific Sidestep Cut.](https://www.ncbi.nlm.nih.gov/pubmed/30986095)
Pradarelli JJ,
Grooms DR,
Hutchison KA,
Onate JA,
Chaudhari AMW,
The American journal of sports medicine
<br>  

* [Arterial Stiffness: Different Metrics, Different Meanings.](https://www.ncbi.nlm.nih.gov/pubmed/30985880)
Humphrey J,
Journal of biomechanical engineering
<br>  

* [Using Machine Learning to Predict Lower-Extremity Injury in US Special Forces.](https://www.ncbi.nlm.nih.gov/pubmed/30985586)
Eagle SR,
Johnson CD,
Flanagan SD,
Mi QI,
Nindl BC,
Medicine and science in sports and exercise
<br>  

import csv
​
myfile =  open('literature_update.csv', 'w')
wr = csv.writer(myfile)
for item in titles:
    wr.writerow([item])